In [3]:
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "J&hP3eOr%cXj8mV3"
NEO4J_DATABASE = "neo4j"   # change if you use another db

from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = NEO4J_URI
AUTH = (NEO4J_USER, NEO4J_PASSWORD)

In [4]:
from neo4j import Result, RoutingControl

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

    result = driver.execute_query(
        "MATCH p=()-[:activates]->() RETURN p LIMIT 25", # need the LLM to write this query based on a natural language query from the user
        database_="neo4j",
        routing_=RoutingControl.READ,
        result_transformer_=Result.graph,
    )

    result

print(
    f"Result query has: {len(result.nodes)} nodes, {len(result.relationships)} relationships"
)

Result query has: 28 nodes, 25 relationships


In [25]:
from neo4j_viz.neo4j import from_neo4j
from neo4j_viz.colors import ColorSpace
import math

def create_visualization_graph(neo4j_result):
    """Create visualization graph with node captions and TUI-based colors"""
    try:
        VG = from_neo4j(neo4j_result)

        for node in VG.nodes:
            # Set caption
            node.caption = node.properties.get("name") or ", ".join(node.labels)

            # Get TUI safely
            tui_val = node.properties.get("tui", None)

            # Handle None or NaN
            if tui_val is None or (isinstance(tui_val, float) and math.isnan(tui_val)):
                tui_val = "unknown"

            # Always convert to string
            tui_val = str(tui_val)

            # If multiple TUIs, pick the first
            if "," in tui_val:
                tui_val = tui_val.split(",")[0].strip()

            # Overwrite back into node properties so color_nodes can see it
            node.properties["tui"] = tui_val

        # Now color all nodes based on tui property
        VG.color_nodes(
            field="tui",
            colors=None,  # default palette
            color_space=ColorSpace.DISCRETE,
            override=True
        )

        return VG.render()
    except Exception as e:
        print(f"Error creating visualization: {str(e)}")
        return None


create_visualization_graph(result)

Error creating visualization: 'Node' object has no attribute 'tui'


In [22]:
VG = from_neo4j(result)

node0 = VG.nodes[0]
print("PROPS:", node0.properties)
print("HAS TUI:", "tui" in node0.properties)
print("TUI value:", node0.properties.get("tui"))
print("Node attributes:", dir(node0))

PROPS: {'cui': 'C0021745', 'name': 'interferon type II', 'description': 'The major interferon produced by mitogenically or antigenically stimulated LYMPHOCYTES. It is structurally different from TYPE I INTERFERON and its major activity is immunoregulation. It has been implicated in the expression of CLASS II HISTOCOMPATIBILITY ANTIGENS in cells that do not normally produce them, leading to AUTOIMMUNE DISEASES.', 'tui': 'T116,T121,T129', 'labels': ['Entity']}
HAS TUI: True
TUI value: T116,T121,T129
Node attributes: ['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_pydantic_core_schema__', '__get_pydantic_json_schema__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__pr